# RPA Workshop 1 - 개인 별 수행

1. 2008.csv 파일을 데이터프레임으로 읽으세요.
- https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi:10.7910/DVN/HG7NV7

2. 해당 데이터는 미국 내 항공 기록입니다 (2008년)

3. Orig와 Dest는 각각 출발지 및 도착지 공항 코드입니다.

4. 위의 데이터프레임에 대해서 다음 내용을 수행하세요.

- 데이터프레임에 대한 결측치 조사

- Dest 공항 코드의 고유한 값별로 빈도 조사

- (선택) Orig와 Dest 값으로 Group By하여 도착 지연 시간의 평균 계산

- flight라는 폴더 만든 후 WD로 지정

- 전체 데이터프레임을 Dest 공항 코드별로 나눠서 엑셀 저장: 공항코드.xlsx 형태



In [ ]:
!unzip 2008.zip

In [15]:
import pandas as pd
df1=pd.read_csv("2008.csv")
df1.isnull().mean()
df1.Dest.unique()
df1.Dest.value_counts()

ATL    136950
ORD    118011
DFW     93077
DEN     78417
LAX     73514
        ...  
TUP         9
AKN         2
OGD         2
CYS         2
DLG         2
Name: Dest, Length: 293, dtype: int64

In [27]:
df1.groupby(['Origin', 'Dest'])["ArrDelay"].mean()

Origin  Dest
ABE     ATL     15.516729
        BHM     -3.000000
        CLE      0.777419
        CLT     -7.975000
        CVG     -1.368421
                  ...    
YUM     IPL      3.613208
        LAS      8.384615
        LAX      2.916667
        PHX      5.653199
        SLC      5.935897
Name: ArrDelay, Length: 4981, dtype: float64

In [24]:
import os

In [25]:
os.getcwd()

'/content'

In [26]:
os.chdir("/content/flight")

In [32]:
for i in df1.Dest.unique():
  df1[df1,Dest==i].to_excel(i+".xlsx")

NameError: ignored

# RPA Workshop 2 - 같은 기관에서 오신 경우 / 관심사가 갖으신 경우 팀으로 수행하셔도 됩니다.

1. 공공데이터포털 또는 AI허브 등에서 관심을 갖는 분야에 대한 표 형태의 엑셀이나 CSV인 경우 행정 데이터를 찾으세요.
- 예: 00현황, 00민원 등에 대한 데이터
- 관심 분야를 찾기 어려우신 경우 첨부된 데이터 들을 확인해보세요.

2. 해당 데이터를 다운로드한 후, 데이터 프레임으로 읽어 보세요.

3. 해당 데이터와 관련된 주요 키워드를 도출한 후, 관련 뉴스를 수집하세요.

4. 수집된 뉴스에서 적정한 개수의 토픽을 발견하세요.

- ***1부터 4의 내용을 학습하신 코드를 이용해서 워드로 만들어서 다운로드 하세요***
- ***워크숍 종료 후 PDF를 공유하는 발표를 진행할 예정입니다***

In [36]:
os.chdir("/content")

In [37]:
import pandas as pd
df=pd.read_csv("소방청_연간화재통계_20211231.csv", encoding="CP949")

In [68]:
from bs4 import BeautifulSoup
from urllib import parse
import requests
import urllib.request
import numpy as np
import pandas as pd
import time

#----------------------------------------------#
#변경할 부분
startdate = "2023.04.22"
enddate = "2023.06.22"
searchword ="화재원인"#검색어
#----------------------------------------------#

base_url = "https://search.naver.com/search.naver?where=news&sm=tab_pge&query"+parse.urlencode([('',searchword)], doseq=True)+"&sort=1&photo=3&field=0&pd=3&ds="+startdate+"&de="+enddate+"&mynews=1&office_type=0&office_section_code=0&news_office_checked=&nso=so:dd,p:from"+startdate.replace(".", "")+"to"+enddate.replace(".", "")+",a:all&mynews=0&start="
titles = [ ]
urls = [ ]
i=1

while True:
 try:
    time.sleep(0.45)
    url = base_url+str(i)
    html = urllib.request.urlopen(url).read()
    soup = BeautifulSoup(html, 'html.parser')
    t_news = soup.select('a.news_tit')  #클래스 표시
    t_len = len(t_news)

    if t_len <1:
        break

    for j in np.arange(t_len)+1:
            try:
                titles.append(soup.select('a.news_tit')[j].text)
                urls.append(soup.select('a.info')[j]['href'])
            except Exception as e:
                continue
    i=i+10

 except Exception as e:
    continue

result = pd.DataFrame({"Url": urls, "Title":titles})
result.Url[0]

'https://n.news.naver.com/mnews/article/654/0000043800?sid=102'

In [69]:
result.shape

(69, 2)

In [70]:
articles = []
dates = []

for i in result.Url:
   try:
    if( "news.naver.com" in i ): #바뀐곳
     print(i)
     time.sleep(0.45)
     html = urllib.request.urlopen(i).read()
     soup = BeautifulSoup(html, 'html.parser')

     news = soup.select('div#dic_area')[0].text #id #, class.
     news_date = soup.select('span.media_end_head_info_datestamp_time._ARTICLE_DATE_TIME')[0].text  #네이버 뉴스의 날짜 시간 부분 지정

     articles.append(news)
     dates.append(news_date.split()[0])

   except Exception as e:
    print(e)
    continue

https://n.news.naver.com/mnews/article/654/0000043800?sid=102
https://n.news.naver.com/mnews/article/015/0004859003?sid=102
https://n.news.naver.com/mnews/article/656/0000054571?sid=102
https://n.news.naver.com/mnews/article/028/0002644876?sid=102
https://n.news.naver.com/mnews/article/658/0000044327?sid=102
https://n.news.naver.com/mnews/article/081/0003369173?sid=102
https://n.news.naver.com/mnews/article/654/0000043219?sid=102
https://n.news.naver.com/mnews/article/654/0000043218?sid=102
https://n.news.naver.com/mnews/article/088/0000820032?sid=102
https://n.news.naver.com/mnews/article/087/0000977063?sid=102
https://n.news.naver.com/mnews/article/015/0004855083?sid=101
https://n.news.naver.com/mnews/article/030/0003106844?sid=105
https://n.news.naver.com/mnews/article/011/0004200327?sid=102
https://n.news.naver.com/mnews/article/025/0003285138?sid=103
https://n.news.naver.com/mnews/article/088/0000817737?sid=102
https://n.news.naver.com/mnews/article/087/0000974086?sid=102
https://

In [71]:
df = pd.DataFrame({'news':articles, 'date':dates})
df

news         date
0   \n주거침입·강도상해 혐의양구의 한 주택에서 80대 노인을 흉기로 다치게 한 50대...  2023.06.22.
1   \n안전보건공단, 스타트업 공모전마인드포지·성심씨엔앰 '최우수'안전보건공단은 21일...  2023.06.21.
2   \n국과수 결과 화재 직접적인 발화원 특정 불가화 가능성 낮지만 배제 않기로…입건자...  2023.06.21.
3   \n국과수 “발화원 특정할수 없다” 경찰, 방화설엔 “가능성 희박” 일축\n\n\n...  2023.06.21.
4   \n오전 9시34분 발화, 연기 자욱헬기와 사다리차로 긴급 구조산모·어린이 등 32...  2023.06.20.
5   \n\n\n\n\n대구 재활용 공장서 원인 모를 화재… 인명 피해는 없어 - 15일...  2023.06.16.
6   \n\n\n\n\n▲ 15일 오전 1시 27분  고성 죽왕면 삼포 오션투유리조트 화...  2023.06.16.
7   \n속초 청년몰 화재에 청년상인 한숨영업장 전소 재개 여부 불투명민자투입 2층 카페...  2023.06.16.
8   \n대구 서구 중리동 재활용 공장에서 대형 화재\n\n\n\n15일 오후 5시 24...  2023.06.15.
9   \n대피 과정에서 2명 중경상·10명 연기 흡입건물 100여㎡ 태우고 2시간여만에 ...  2023.06.15.
10  \n으뜸 중기파이터코리아 '세이브에스'열 감지해 소화제 자동 분사\n\n\n\n콘센...  2023.06.12.
11  \n\n\n\n\n남화영 소방청장미래학자인 레이 커즈와일은 ‘기술의 진화 속도가 무...  2023.06.11.
12  \n\n\n\n\n[서울경제] 서울시는 새로운 행정 수요와 사회 변화에 선제적으로 ...  2023.06.11.
13  \n  \n\n\n\n영화 ‘노트르담 온 파이어’의 화재 진압 장면. 대성당 첨탑 ...  2023.06.07.
14  \n다행히 인명피해 없어…소방당국 "정확한 화재 원인 조사 중"\n\n\n\n4일 ...  2023.06.04.
15  \n최근 5년간 도내 전동킥보드 화재 사고 13건전동킥보드 화재 주요 원인 ‘과충전...  2023.06.01.
16  \n법원 "화덕 불씨 화재 원인으로 단정할 수 없어"화덕 불씨 취급 부주의로 불을 ...  2023.05.31.
17  \n\n\n\n\n삼성전자서비스 엔지니어가 에어컨 실외기를 점검하고 있다. 삼성전자...  2023.05.29.
18  \n인명피해는 없어\n\n\n\n28일 오전 4시52분쯤 영천시 야사동 한 아파트 ...  2023.05.28.
19  \n■국내 첫 상용화 '에이치투' 연산 330㎿h 계룡공장 준공바나듐 원료 활용한 ...  2023.05.26.
20  \n배터리 폭발 때 대피로 확보 곤란울산 등서 인명·재산 피해 급증소방 “반드시 야...  2023.05.21.
21  \n소방청 통계 2018년 5건→2022년 115건\n\n\n\n전동킥보드 [연합뉴...  2023.05.21.
22  \n\t\t\t【평창】지난 3월 경기 안산시의 한 다가구주택에서 전기적 요인으로 인...  2023.05.18.
23  \n\n\n\n\n18일 발생한 성환읍 한 제조업체 화재 모습. 사진=독자 제공 충...  2023.05.18.
24  \n임계 온도 넘어가면 경보 작동측정거리 일반 제품의 10배충전소·軍시설 등 공급 ...  2023.05.17.
25  \n세탁기·TV 이어… 에어컨 화재원인·점유율 두고 ‘3차 대전’\t\t\t\t\t...  2023.05.15.
26  \n평창서 주차된 전기승용차서 화재 발생…차량 전소횡성 소사리서 주택 화재 발생해 ...  2023.05.15.
27  \n충남소방, 7월까지 77곳 안전 점검\n\n\n\n펜션에서 발생한 화재를 소방관...  2023.05.14.
28  \n한국타이어 대전공장 두달째 가동 중단안전 문제·여론 눈치에 재가동 ‘미지수’전소...  2023.05.11.
29  \n\n\n\n\n◇11일 낮 12시18분께 삼척시 미로면 상사전리 도로에서 운행중...  2023.05.11.
30  \n영월·고성·양구 등 곳곳 산불 속출강원도내 산지와 영동지역을 중심으로 건조주의보...  2023.05.04.
31  \n영월 쌍용리와 예밀리 사유림에서 화재 발생고성과 양구에서도 산불 속출…원인 조사...  2023.05.03.
32  \n인근 빌라 주민 1명 연기 흡입…인명피해는 없어\n\n\n\n3일 오후 3시 1...  2023.05.03.
33  \n\n\n\n\n고성소방서(서장:서강원)은 3일 서내 소회의실에서 관내 숙박시설 ...  2023.05.03.
34  \n부산진구 아파트서 심야사고불 번지며 옆 차량 5대 파손침수조 활용 등 진압 어려...  2023.05.01.
35  \n\n\n\n\n김항수 대전경찰청 과학수사대장이 26일 한국타이어 대전공장에서 진...  2023.04.26.
36  \n최근 5년간 천안·아산 50건화학적 요인 34% 차지…야적 특성에 진압 어려워\...  2023.04.25.
37  \n“충전시설 관련 소방설비 설치기준, 법률로 정해야”\n\n\n\n경북 경주시 율...  2023.04.25.

In [72]:
df.to_csv("news2023.csv", index=False)

In [73]:
!pip install konlpy
!pip install tqdm #진행상태를 알려주는 기능 제공

import pandas as pd
import matplotlib.pyplot as plt
import urllib.request
from konlpy.tag import Okt, Kkma
from tqdm import tqdm

print(df.isnull().values.any()) #null값이 있는지 체크
train_data = df.dropna(how = 'any') # Null 값이 존재하는 행 제거
print(train_data.isnull().values.any()) # Null 값이 존재하는지 확인

train_data['news'] = train_data['news'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","") #한글만 남김

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
False
False


<ipython-input-73-405728fb263c>:14: FutureWarning: The default value of regex will change from True to False in a future version.
  train_data['news'] = train_data['news'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","") #한글만 남김


In [74]:
train_data.news[1]
train_data.shape

(38, 2)

In [76]:
#값을 저장할 비어있는 리스트를 만듬.
tokenized_data = []

# 불용어 정의
stopwords = ['을', '등', '것', '년', '수', '분기','위','해','월','일',
             '부터', '까지' '적', '의','가','이','은',
             '들','는','좀','잘','걍','과','도','를','으로',
             '자','에','와','한','하다', '점', '나']

# 형태소 분석기 OKT를 사용한 토큰화 작업 (다소 시간 소요)
okt = Kkma() #Kkma()

for sentence in tqdm(train_data['news']): #리뷰 각각에 대해서 아래작업 반복
    tokenized_sentence = okt.nouns(sentence)  #각 리뷰에서 명사만 뽑아서 리스트
    #위에서 명사만 추출한 결과에서, 각 명사가 stopwords인지 체크하고
    #stopwords 아닌 명사만 사용함
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    tokenized_data.append(stopwords_removed_sentence)

len(tokenized_data)

100%|██████████| 38/38 [00:21<00:00,  1.74it/s]


38

In [81]:
from gensim import corpora
dictionary = corpora.Dictionary(tokenized_data) #사전 구성
corpus = [dictionary.doc2bow(text) for text in tokenized_data]

import gensim
NUM_TOPICS = 15
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS,
                                           id2word=dictionary, passes=15)

#각 토픽의 출현확률이 높은 단어를 지정된 갯수만큼 반환
topics = ldamodel.print_topics(num_words=5)

#위 내용을 출력
for topic in topics:
    print(topic)

(0, '0.005*"만" + 0.005*"대" + 0.005*"여명" + 0.004*"현장" + 0.004*"상태"')
(1, '0.005*"화재" + 0.005*"발생" + 0.005*"원인" + 0.005*"소방" + 0.004*"제품"')
(2, '0.004*"발생" + 0.004*"원인" + 0.004*"설명" + 0.004*"제공" + 0.004*"실시"')
(3, '0.007*"원인" + 0.007*"화재" + 0.006*"소방" + 0.006*"시" + 0.006*"발생"')
(4, '0.008*"화재" + 0.007*"면" + 0.007*"원인" + 0.006*"씨" + 0.006*"당국"')
(5, '0.000*"원인" + 0.000*"발생" + 0.000*"화재" + 0.000*"개" + 0.000*"중"')
(6, '0.004*"단독" + 0.004*"지난해" + 0.004*"원주" + 0.004*"지원" + 0.004*"원"')
(7, '0.005*"파악" + 0.005*"여" + 0.005*"인명" + 0.005*"피해" + 0.005*"불이"')
(8, '0.004*"속초" + 0.004*"만" + 0.004*"대" + 0.004*"시" + 0.002*"지역"')
(9, '0.007*"동해" + 0.006*"동" + 0.005*"중" + 0.003*"포" + 0.003*"투"')
(10, '0.007*"시" + 0.005*"소방" + 0.005*"발생" + 0.005*"조사" + 0.005*"원인"')
(11, '0.009*"컨설팅" + 0.009*"빅" + 0.009*"곳" + 0.009*"소방시설" + 0.008*"가능성"')
(12, '0.004*"개" + 0.004*"내" + 0.004*"만개" + 0.002*"만" + 0.002*"화재"')
(13, '0.005*"인명" + 0.005*"만" + 0.005*"피해" + 0.005*"건" + 0.004*"소방"')
(14, '0.000*"원인" + 0.000*"화재" + 0.0

In [82]:
from gensim.models import Word2Vec

model = Word2Vec(sentences = tokenized_data)

In [87]:
print(model.wv.most_similar("화재"))
print(model.wv.most_similar("소방"))

[('명', 0.486883282661438), ('중', 0.43877264857292175), ('차', 0.4327608346939087), ('불꽃', 0.42998212575912476), ('불이', 0.4279463589191437), ('감식', 0.4239436388015747), ('승용차', 0.40141749382019043), ('불', 0.3975144922733307), ('확인', 0.3940817713737488), ('간', 0.38462749123573303)]
[('장소', 0.39702022075653076), ('방청', 0.39066827297210693), ('병원', 0.3859691321849823), ('만', 0.3850264549255371), ('명', 0.37891140580177307), ('점검', 0.37583476305007935), ('전체', 0.35951992869377136), ('승용차', 0.35916224122047424), ('배터리', 0.34743794798851013), ('규모', 0.3399624824523926)]


In [90]:
!pip install python-docx

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [91]:
from docx import Document

doc = Document()
#스타일 적용하기 (일반 텍스트, 글꼴은 ‘맑은고딕체’)
from docx.oxml.ns import qn
style = doc.styles['Normal']
style.font.name =  '맑은 고딕'
style._element.rPr.rFonts.set(qn('w:eastAsia'), '맑은 고딕')

#제목 넣기
doc.add_heading('워크샵 보고서, H1', level=1) #level: 0~6

#문단 넣기
doc.add_paragraph()
doc.paragraphs[0].text = "워크샵 2번 문제"

doc.add_paragraph()
doc.paragraphs[1].text = "1. 데이터셋이름:  소방청_연간화재통계_20211231.csv"

doc.add_paragraph()
doc.paragraphs[2].text = "2. 데이터셋 컬럼 정보:  컬럼 정보를 넣으세요"

doc.add_paragraph()
doc.paragraphs[3].text = "3. 주요 키워드: 화재원인"

doc.add_paragraph()
doc.paragraphs[4].text = "4. 수집된 뉴스 건수: 69건"

doc.add_paragraph()
doc.paragraphs[5].text = "5. 토픽 개수: 3"

doc.add_paragraph()
doc.paragraphs[6].text = "6. 담배, 전기차, 기기 "

#이미지 추가
#doc.add_picture('이미지이름.jpg',width= Cm(16), height= Cm(9))

#표 추가
table = doc.add_table(rows = 3, cols = 2)
table.style = doc.styles['Table Grid']  #기본 스타일

#표 내용 채우기

#첫 행
row1 = table.rows[0].cells
# 첫 행의 각 열들에 접근해서 값 입력
row1[0].text = '담배'
row1[1].text = '쓰레기가 많은 곳이나 낙엽이 쌓인 산 등에서 담배를 피우고 난 후 버린 담뱃불이 화재의 원인으로 가장 많이 등장함'

#두번째 행
row2 = table.rows[1].cells
# 두번째 행의 각 열들에 접근해서 값 입력
row2[0].text = '전기차'
row2[1].text = '전기차 충전 중에 발생하는 폭발과 화재가 최근 화재 원인으로 떠오르고 있음'

#세번째 행
row3 = table.rows[2].cells
# 두번째 행의 각 열들에 접근해서 값 입력
row3[0].text = '기기'
row3[1].text = '전자 기기의 과열이나 사용 부주의 등으로 인한 화재가 매년 꾸준하게 발생하고 있음'


#저장
doc.save('테스트.docx')